# 🌟 Influencer Scoring Model

## Social Media ROI Attribution & Influencer Performance Analyzer

This notebook builds a composite influencer effectiveness score (0-100) based on:
- Engagement Quality Score (25%)
- Audience Authenticity (25%)
- Conversion Rate (30%)
- Cost Efficiency/ROI (15%)
- Brand Alignment (5%)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries loaded!")

In [ ]:
# Load data
data_dir = Path("../data/raw")

influencers = pd.read_csv(data_dir / "influencers.csv")
posts = pd.read_csv(data_dir / "posts.csv")
conversions = pd.read_csv(data_dir / "conversions.csv")

print(f"📊 Loaded {len(influencers):,} influencers, {len(posts):,} posts, {len(conversions):,} conversions")

---
## 1. Calculate Component Scores

In [ ]:
# Aggregate post metrics by influencer
inf_post_metrics = posts.groupby('influencer_id').agg({
    'post_id': 'count',
    'likes': 'sum',
    'comments': 'sum',
    'saves': 'sum',
    'shares': 'sum',
    'reach': 'sum',
    'impressions': 'sum',
    'is_sponsored': 'sum'
}).rename(columns={'post_id': 'total_posts', 'is_sponsored': 'sponsored_posts'})

# Merge with influencer data
inf_data = influencers.merge(inf_post_metrics, on='influencer_id', how='left')

print(f"📊 Merged data for {len(inf_data):,} influencers")
inf_data.head()

In [ ]:
# Add conversion data
inf_conversions = conversions[conversions['influencer_id'].notna()].groupby('influencer_id').agg({
    'conversion_id': 'count',
    'order_value': 'sum'
}).rename(columns={'conversion_id': 'conversions', 'order_value': 'revenue'})

inf_data = inf_data.merge(inf_conversions, on='influencer_id', how='left')
inf_data['conversions'] = inf_data['conversions'].fillna(0)
inf_data['revenue'] = inf_data['revenue'].fillna(0)

print(f"📊 Added conversion data")
print(f"   Influencers with conversions: {(inf_data['conversions'] > 0).sum():,}")

### 1.1 Engagement Quality Score (25%)

In [ ]:
# Engagement quality = weighted by save rate (high intent) vs just likes
# Saves are 3x more valuable than likes, Comments 2x
inf_data['weighted_engagement'] = (
    inf_data['likes'] * 1 + 
    inf_data['comments'] * 2 + 
    inf_data['saves'] * 3 + 
    inf_data['shares'] * 2
)

# Calculate quality score (weighted engagement per 1000 followers)
inf_data['engagement_quality'] = inf_data['weighted_engagement'] / (inf_data['follower_count'] / 1000)

# Normalize to 0-100
scaler = MinMaxScaler(feature_range=(0, 100))
inf_data['engagement_quality_score'] = scaler.fit_transform(inf_data[['engagement_quality']].fillna(0))

print("✅ Engagement Quality Score calculated")
print(f"   Mean: {inf_data['engagement_quality_score'].mean():.2f}")
print(f"   Median: {inf_data['engagement_quality_score'].median():.2f}")

### 1.2 Audience Authenticity Score (25%)

In [ ]:
# Already have authenticity score from data generation (0-1)
inf_data['authenticity_score'] = inf_data['audience_authenticity_score'] * 100

print("✅ Authenticity Score calculated")
print(f"   Mean: {inf_data['authenticity_score'].mean():.2f}")
print(f"   Median: {inf_data['authenticity_score'].median():.2f}")

### 1.3 Conversion Rate Score (30%)

In [ ]:
# Conversion rate = conversions / sponsored posts
inf_data['conversion_rate'] = np.where(
    inf_data['sponsored_posts'] > 0,
    inf_data['conversions'] / inf_data['sponsored_posts'],
    0
)

# Normalize to 0-100
inf_data['conversion_score'] = scaler.fit_transform(inf_data[['conversion_rate']].fillna(0))

print("✅ Conversion Score calculated")
print(f"   Mean: {inf_data['conversion_score'].mean():.2f}")
print(f"   Median: {inf_data['conversion_score'].median():.2f}")

### 1.4 Cost Efficiency / ROI Score (15%)

In [ ]:
# ROI = (Revenue - Cost) / Cost
inf_data['total_cost'] = inf_data['avg_collaboration_cost'] * inf_data['sponsored_posts'].fillna(0)
inf_data['roi'] = np.where(
    inf_data['total_cost'] > 0,
    (inf_data['revenue'] - inf_data['total_cost']) / inf_data['total_cost'],
    0
)

# Cap extreme ROI values
inf_data['roi_capped'] = inf_data['roi'].clip(-1, 10)

# Normalize to 0-100
inf_data['roi_score'] = scaler.fit_transform(inf_data[['roi_capped']].fillna(0))

print("✅ ROI Score calculated")
print(f"   Mean: {inf_data['roi_score'].mean():.2f}")
print(f"   Median: {inf_data['roi_score'].median():.2f}")

### 1.5 Brand Alignment Score (5%)

In [ ]:
# Brand alignment based on content category popularity and engagement consistency
# Higher for fashion-relevant categories
alignment_scores = {
    'Luxury Fashion': 95,
    'Streetwear': 85,
    'Sustainable Fashion': 90,
    'Fast Fashion': 80,
    'Accessories': 85,
    'Footwear': 82,
    'Activewear': 78,
    'Vintage/Thrift': 88
}

inf_data['brand_alignment_score'] = inf_data['content_category'].map(alignment_scores).fillna(75)

print("✅ Brand Alignment Score calculated")
print(f"   Mean: {inf_data['brand_alignment_score'].mean():.2f}")

---
## 2. Calculate Composite Influencer Score

In [ ]:
# Weight formula
# Influencer_Score = (0.25 * Engagement) + (0.25 * Authenticity) + (0.30 * Conversion) + (0.15 * ROI) + (0.05 * Alignment)

inf_data['influencer_score'] = (
    0.25 * inf_data['engagement_quality_score'] +
    0.25 * inf_data['authenticity_score'] +
    0.30 * inf_data['conversion_score'] +
    0.15 * inf_data['roi_score'] +
    0.05 * inf_data['brand_alignment_score']
)

print("✅ Composite Influencer Score calculated!")
print(f"\n   Mean Score: {inf_data['influencer_score'].mean():.2f}")
print(f"   Median Score: {inf_data['influencer_score'].median():.2f}")
print(f"   Min: {inf_data['influencer_score'].min():.2f}")
print(f"   Max: {inf_data['influencer_score'].max():.2f}")

In [ ]:
# Segment influencers
def segment_influencer(score):
    if score >= 75:
        return 'High Performer'
    elif score >= 50:
        return 'Medium Performer'
    else:
        return 'Low Performer'

inf_data['performance_segment'] = inf_data['influencer_score'].apply(segment_influencer)

segment_counts = inf_data['performance_segment'].value_counts()
print("\n📊 Influencer Segments:")
for segment, count in segment_counts.items():
    pct = count / len(inf_data) * 100
    print(f"   {segment}: {count:,} ({pct:.1f}%)")

---
## 3. Visualize Influencer Scores

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Score distribution
axes[0, 0].hist(inf_data['influencer_score'], bins=50, color='steelblue', edgecolor='white', alpha=0.8)
axes[0, 0].axvline(x=75, color='green', linestyle='--', label='High Performer Threshold')
axes[0, 0].axvline(x=50, color='orange', linestyle='--', label='Medium Performer Threshold')
axes[0, 0].set_title('Influencer Score Distribution', fontweight='bold', fontsize=14)
axes[0, 0].set_xlabel('Influencer Score')
axes[0, 0].set_ylabel('Count')
axes[0, 0].legend()

# Scores by Tier
tier_order = ['nano', 'micro', 'mid', 'macro', 'mega']
sns.boxplot(data=inf_data, x='tier', y='influencer_score', order=tier_order, ax=axes[0, 1], palette='RdYlGn')
axes[0, 1].set_title('Influencer Score by Tier', fontweight='bold', fontsize=14)
axes[0, 1].set_xlabel('Tier')
axes[0, 1].set_ylabel('Influencer Score')

# Segment distribution
segment_order = ['High Performer', 'Medium Performer', 'Low Performer']
colors = ['green', 'orange', 'red']
segment_data = inf_data['performance_segment'].value_counts().reindex(segment_order)
axes[1, 0].pie(segment_data.values, labels=segment_data.index, autopct='%1.1f%%', colors=colors)
axes[1, 0].set_title('Influencer Performance Segments', fontweight='bold', fontsize=14)

# Score components heatmap
components = ['engagement_quality_score', 'authenticity_score', 'conversion_score', 'roi_score', 'brand_alignment_score']
component_names = ['Engagement', 'Authenticity', 'Conversion', 'ROI', 'Brand Alignment']
tier_component = inf_data.groupby('tier')[components].mean().reindex(tier_order)
tier_component.columns = component_names
sns.heatmap(tier_component, annot=True, fmt='.1f', cmap='RdYlGn', ax=axes[1, 1])
axes[1, 1].set_title('Score Components by Tier', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.savefig('../data/influencer_scoring.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 4. Top & Bottom Influencers

In [ ]:
# Top 10 influencers
display_cols = ['username', 'platform', 'tier', 'follower_count', 'engagement_rate', 
                'influencer_score', 'conversions', 'revenue', 'avg_collaboration_cost']

print("🏆 TOP 10 INFLUENCERS")
print("="*80)
top_10 = inf_data.nlargest(10, 'influencer_score')[display_cols]
print(top_10.to_string(index=False))

In [ ]:
# Bottom 10 influencers (candidates for removal)
print("\n⚠️ BOTTOM 10 INFLUENCERS (Consider Removal)")
print("="*80)
bottom_10 = inf_data.nsmallest(10, 'influencer_score')[display_cols]
print(bottom_10.to_string(index=False))

---
## 5. ROI Analysis by Segment

In [ ]:
# Calculate segment metrics
segment_metrics = inf_data.groupby('performance_segment').agg({
    'influencer_id': 'count',
    'total_cost': 'sum',
    'revenue': 'sum',
    'conversions': 'sum',
    'influencer_score': 'mean'
}).rename(columns={'influencer_id': 'count'})

segment_metrics['roi'] = (segment_metrics['revenue'] - segment_metrics['total_cost']) / segment_metrics['total_cost'] * 100
segment_metrics['cost_per_conversion'] = segment_metrics['total_cost'] / segment_metrics['conversions']

segment_metrics = segment_metrics.reindex(segment_order)

print("📊 Performance Segment Analysis")
print(segment_metrics.round(2))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROI by segment
colors = ['green', 'orange', 'red']
segment_metrics['roi'].plot(kind='bar', ax=axes[0], color=colors)
axes[0].set_title('ROI by Performance Segment', fontweight='bold', fontsize=14)
axes[0].set_ylabel('ROI (%)')
axes[0].tick_params(axis='x', rotation=45)
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Cost per conversion by segment
segment_metrics['cost_per_conversion'].plot(kind='bar', ax=axes[1], color=colors)
axes[1].set_title('Cost per Conversion by Segment', fontweight='bold', fontsize=14)
axes[1].set_ylabel('Cost ($)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../data/segment_roi.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 6. Recommendations

In [ ]:
print("="*60)
print("💡 STRATEGIC RECOMMENDATIONS")
print("="*60)

# High performers
high_performers = inf_data[inf_data['performance_segment'] == 'High Performer']
print(f"\n🏆 HIGH PERFORMERS ({len(high_performers):,} influencers)")
print("   → Action: Increase investment, pursue long-term partnerships")
print(f"   → Avg ROI: {high_performers['roi'].mean()*100:.1f}%")

# Medium performers
medium_performers = inf_data[inf_data['performance_segment'] == 'Medium Performer']
print(f"\n⚡ MEDIUM PERFORMERS ({len(medium_performers):,} influencers)")
print("   → Action: Optimize content strategy, A/B test different approaches")
print(f"   → Potential uplift if moved to high performer: ~{(75 - medium_performers['influencer_score'].mean()):.1f} points")

# Low performers
low_performers = inf_data[inf_data['performance_segment'] == 'Low Performer']
print(f"\n⚠️ LOW PERFORMERS ({len(low_performers):,} influencers)")
print("   → Action: Reduce investment or terminate partnerships")
potential_savings = low_performers['total_cost'].sum()
print(f"   → Potential savings: ${potential_savings:,.2f}")

# Micro vs Macro insight
micro = inf_data[inf_data['tier'] == 'micro']
macro = inf_data[inf_data['tier'] == 'macro']
print(f"\n📈 MICRO vs MACRO INSIGHT")
print(f"   Micro avg score: {micro['influencer_score'].mean():.1f}")
print(f"   Macro avg score: {macro['influencer_score'].mean():.1f}")
if micro['influencer_score'].mean() > macro['influencer_score'].mean():
    print("   → Recommendation: Shift budget from macro to micro-influencers")

---
## 7. Save Scored Data

In [ ]:
# Save scored influencer data
output_cols = ['influencer_id', 'username', 'platform', 'tier', 'follower_count', 
               'engagement_rate', 'audience_authenticity_score', 'avg_collaboration_cost',
               'total_posts', 'sponsored_posts', 'conversions', 'revenue',
               'engagement_quality_score', 'authenticity_score', 'conversion_score', 
               'roi_score', 'brand_alignment_score', 'influencer_score', 'performance_segment']

output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

inf_data[output_cols].to_csv(output_dir / "influencer_scores.csv", index=False)

print("💾 Saved scored influencer data to ../data/processed/influencer_scores.csv")
print(f"\n📊 Summary:")
print(f"   Total influencers scored: {len(inf_data):,}")
print(f"   High performers: {len(high_performers):,} ({len(high_performers)/len(inf_data)*100:.1f}%)")
print(f"   Medium performers: {len(medium_performers):,} ({len(medium_performers)/len(inf_data)*100:.1f}%)")
print(f"   Low performers: {len(low_performers):,} ({len(low_performers)/len(inf_data)*100:.1f}%)")

---
## ✅ Influencer Scoring Complete!

**Outputs:**
- Composite influencer score (0-100)
- Performance segments (High/Medium/Low)
- Strategic recommendations
- `influencer_scores.csv` for dashboard integration

**Charts saved:**
- `influencer_scoring.png`
- `segment_roi.png`

**Next: Build interactive dashboard with Streamlit!**